In [1]:
import optuna
import plotly 
import sqlite3

STUDY_DB_PATH = "sqlite:///../optuna_studies/tuning.db"


conn = sqlite3.connect("../optuna_studies/tuning.db")
cursor = conn.cursor()


cursor.execute("SELECT study_name FROM studies")
study_names = cursor.fetchall()


available_studies = [name[0] for name in study_names]
available_studies


/Users/sergiocarcamo/Dev/thesis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['RandomForest_selected_features_study',
 'XGBoost_selected_features_study',
 'XGBoostQuantile_selected_features_study',
 'LightGBM_selected_features_study',
 'AdaBoost_selected_features_study',
 'XGBoost_rfe_100.json_study',
 'XGBoost_rfe_100_study',
 'XGBoostQuantile_rfe_100_study',
 'LightGBM_rfe_100_study',
 'AdaBoost_rfe_100_study',
 'RandomForest_rfe_100_study',
 'ElasticNet_rfe_100_study',
 'DecisionTree_rfe_100_study',
 'XGBoost_rfe_100_non_pca_study',
 'XGBoostQuantile_rfe_100_non_pca_study',
 'LightGBM_rfe_100_non_pca_study',
 'RandomForest_rfe_100_non_pca_study',
 'LightGBM_rfecv_36_pca_scaled_study']

In [11]:
STUDY_NAME = "LightGBM_rfe_100_non_pca_study" 
model_name = STUDY_NAME.split("_")[0]
#STUDY_NAME = "XGBoostQuantile_rfe_100_non_pca_study" 

loaded_study = optuna.load_study(study_name=STUDY_NAME, storage=STUDY_DB_PATH)

print(f"Loaded study '{STUDY_NAME}' with {len(loaded_study.trials)} trials.")
print("Best value:", loaded_study.best_value)

TARGET_NAME = "mape"


Loaded study 'LightGBM_rfe_100_non_pca_study' with 100 trials.
Best value: 0.058749287504529936


In [13]:

fig1 = optuna.visualization.plot_optimization_history(loaded_study, target_name=TARGET_NAME)

current_title = fig1.layout.title.text if fig1.layout.title and fig1.layout.title.text else ""
fig1.update_layout(title_text=f"{current_title} - {model_name}", title_x=0.5)

fig1.show()


In [14]:
fig2 = optuna.visualization.plot_parallel_coordinate(loaded_study, target_name=TARGET_NAME)

current_title = fig2.layout.title.text if fig2.layout.title and fig2.layout.title.text else ""
fig2.update_layout(title_text=f"{current_title} - {model_name}", title_x=0.5)

fig2.show()


[W 2025-05-28 11:12:23,835] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,840] Trial 7 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,841] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,841] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,841] Trial 14 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,841] Trial 15 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,842] Trial 19 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,842] Trial 45 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,843] Trial 47 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:23,843] Tria

In [15]:

fig3 = optuna.visualization.plot_param_importances(loaded_study)

current_title = fig3.layout.title.text if fig3.layout.title and fig3.layout.title.text else ""
fig3.update_layout(title_text=f"{current_title} - {model_name}", title_x=0.5)

fig3.show()



[W 2025-05-28 11:12:43,516] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,517] Trial 7 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,517] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,518] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,518] Trial 14 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,518] Trial 15 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,519] Trial 19 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,519] Trial 45 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,520] Trial 47 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:12:43,520] Tria

In [6]:

fig4 = optuna.visualization.plot_slice(loaded_study)
fig4.show()


[W 2025-05-28 11:09:14,462] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,463] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,464] Trial 16 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,464] Trial 18 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,465] Trial 27 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,465] Trial 29 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,466] Trial 34 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,466] Trial 35 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,467] Trial 37 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,467] Tr

In [7]:

fig5 = optuna.visualization.plot_contour(loaded_study, params=['model__reg_alpha', 'model__reg_lambda'])
fig5.show()


[W 2025-05-28 11:09:14,595] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,596] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,597] Trial 16 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,597] Trial 18 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,598] Trial 27 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,598] Trial 29 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,599] Trial 34 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,599] Trial 35 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,599] Trial 37 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,600] Tr

In [8]:
try:
    fig5 = optuna.visualization.plot_contour(loaded_study, params=['model__learning_rate', 'model__max_depth'])
    fig5.show()
except Exception as e:
    print(f"Could not generate contour plot: {e}") 


[W 2025-05-28 11:09:14,651] Trial 5 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,651] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,652] Trial 16 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,652] Trial 18 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,653] Trial 27 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,654] Trial 29 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,654] Trial 34 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,655] Trial 35 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,655] Trial 37 is omitted in visualization because its objective value is inf or nan.
[W 2025-05-28 11:09:14,656] Tr

In [16]:
trials_df = loaded_study.trials_dataframe()
trials_df.head()

,number,value,datetime_start,datetime_complete,duration,params_model__colsample_bytree,params_model__learning_rate,params_model__max_depth,params_model__min_child_samples,params_model__n_estimators,params_model__num_leaves,params_model__reg_alpha,params_model__reg_lambda,params_model__subsample,state
0,0,0.063488,2025-05-26 06:21:18.751458,2025-05-26 06:22:25.054258,0 days 00:01:06.302800,0.577824,0.044199,21,28,900,79,0.140340,22.107817,0.758981,COMPLETE
1,1,0.061451,2025-05-26 06:22:25.177855,2025-05-26 06:23:31.437596,0 days 00:01:06.259741,0.664037,0.136693,23,49,850,94,0.002024,0.211527,0.935150,COMPLETE
2,2,0.072667,2025-05-26 06:23:31.543629,2025-05-26 06:24:38.230966,0 days 00:01:06.687337,0.847230,0.011579,22,33,1350,37,0.311858,1.418841,0.811295,COMPLETE
3,3,0.063116,2025-05-26 06:24:38.340979,2025-05-26 06:26:44.904656,0 days 00:02:06.563677,0.765553,0.014341,29,41,1150,133,0.012075,0.003131,0.801671,COMPLETE
4,4,0.070270,2025-05-26 06:26:45.009552,2025-05-26 06:27:33.111382,0 days 00:00:48.101830,0.707227,0.017509,7,40,650,113,0.097494,0.580552,0.925256,COMPLETE
